In [ ]:
import os, sys, torch
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from multilayer_model.m3_hybrid_final import HierarchicalHybridCVAE3
from multilayer_loss.l_multi3_hybrid_final import l_multi3_hybrid_final
from vae_earlystopping import EarlyStopping

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# 데이터 로드
x1_raw = np.load('../data/metal.npy')
x2_raw = np.load('../data/support_norm.npy')
x3_raw = np.load('../data/pre_fin.npy')
c_raw = np.load('../data/re_fin.npy')

# 8:1:1 분할
tr_idx, te_idx = train_test_split(np.arange(len(x1_raw)), test_size=0.1, random_state=42)
tr_idx, va_idx = train_test_split(tr_idx, test_size=0.1, random_state=42)

sc1, sc2, sc3, scc = MinMaxScaler(), MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
def prep(d, i, s, f=False): return s.fit_transform(d[i]) if f else s.transform(d[i])

x1_tr, x1_va, x1_te = prep(x1_raw, tr_idx, sc1, True), prep(x1_raw, va_idx, sc1), prep(x1_raw, te_idx, sc1)
x2_tr, x2_va, x2_te = prep(x2_raw, tr_idx, sc2, True), prep(x2_raw, va_idx, sc2), prep(x2_raw, te_idx, sc2)
x3_tr, x3_va, x3_te = prep(x3_raw, tr_idx, sc3, True), prep(x3_raw, va_idx, sc3), prep(x3_raw, te_idx, sc3)
c_tr, c_va, c_te = prep(c_raw, tr_idx, scc, True), prep(c_raw, va_idx, scc), prep(c_raw, te_idx, scc)

def to_t(a): return torch.tensor(a, dtype=torch.float32)
train_loader = DataLoader(TensorDataset(to_t(x1_tr), to_t(x2_tr), to_t(x3_tr), to_t(c_tr)), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(to_t(x1_va), to_t(x2_va), to_t(x3_va), to_t(c_va)), batch_size=64)
test_loader = DataLoader(TensorDataset(to_t(x1_te), to_t(x2_te), to_t(x3_te), to_t(c_te)), batch_size=64)

x_dims = [x1_tr.shape[1], x2_tr.shape[1], x3_tr.shape[1]]

In [ ]:
def run_training(mode='mse'):
    model = HierarchicalHybridCVAE3(x_dims, c_tr.shape[1], [16, 8, 4]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    es = EarlyStopping(patience=50)
    
    history = []
    for epoch in range(1, 801):
        model.train()
        t_loss = 0
        for b1, b2, b3, bc in train_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            optimizer.zero_grad()
            out = model(b1, b2, b3, bc)
            res = l_multi3_hybrid_final(out['recons'], out['logits'], [b1, b2, b3], out['mus'], out['lvs'], epoch, 800)
            loss = (res['bce'] if mode=='bce' else res['mse']) + res['kl']
            loss.backward(); optimizer.step(); t_loss += loss.item()
            
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v1, v2, v3, vc in val_loader:
                v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
                vo = model(v1, v2, v3, vc)
                vr = l_multi3_hybrid_final(vo['recons'], vo['logits'], [v1, v2, v3], vo['mus'], vo['lvs'], epoch, 800)
                v_loss += (vr['bce'] if mode=='bce' else vr['mse']).item()
        
        avg_v = v_loss/len(val_loader)
        history.append(avg_v)
        if epoch % 100 == 0: print(f"Mode {mode} | Epoch {epoch} | Val Loss: {avg_v:.6f}")
        if es(avg_v, model): break
    es.load_best_model(model)
    return model, history

print("Training Stage 1: BCE...")
m_bce, h_bce = run_training('bce')
print("Training Stage 2: MSE...")
m_mse, h_mse = run_training('mse')

In [ ]:
m_bce.eval(); m_mse.eval()
preds, trues = [], []
with torch.no_grad():
    for b1, b2, b3, bc in test_loader:
        bc = bc.to(device)
        p1, p2, p3 = m_bce.sample(bc, device)
        _, v2, _ = m_mse.sample(bc, device)
        
        # BCE Prob * MSE Value
        combined = (p2 * v2).cpu().numpy()
        preds.append(sc2.inverse_transform(combined))
        trues.append(sc2.inverse_transform(b2.numpy()))

y_p, y_t = np.concatenate(preds).flatten(), np.concatenate(trues).flatten()
print(f"\n🏆 Final Single Run R2 Score: {r2_score(y_t, y_p):.4f}")

plt.figure(figsize=(10, 4))
plt.scatter(y_t, y_p, alpha=0.5, color='purple')
plt.plot([min(y_t), max(y_t)], [min(y_t), max(y_t)], 'r--')
plt.title("Predicted vs Actual Catalyst Composition")
plt.xlabel("Actual Value"); plt.ylabel("Predicted Value")
plt.show()